# Demo SEA data calculations
This demonstrates the use of low-level routines to replicate signal analysis performed by SEA.

Each analysis product is returned as a `xarray.DataArray`. At the end, they are merged into a single `xarray.Dataset` containing all all the different data results along with their labels, coordinates, and metadata.

In [ ]:
from matplotlib import pyplot as plt
import channel_analysis
from channel_analysis import figures
import iqwaveform

plt.style.use('iqwaveform.ieee_double_column')

dataset = channel_analysis.load('20241122-11h25m59.zarr.zip').set_xindex(
    ['antenna_name', 'center_frequency']
)
dataset

In [ ]:
fc = 3750e6
antenna_name = 'Reference'

cyclic_power = dataset.cyclic_channel_power.sel(
    antenna_name=antenna_name, center_frequency=fc
)
figures.plot_cyclic_channel_power(cyclic_power)

plt.subplots()
(
    dataset.cellular_cyclic_autocorrelation.sel(
        link_direction='downlink', subcarrier_spacing=30e3, drop=True
    )
    .sel(center_frequency=fc)
    .plot.line(x='cyclic_sample_lag')
)
plt.subplots()
(
    dataset.persistence_spectrum.sel(
        center_frequency=fc,
        persistence_statistic=['mean', 'max'],
        antenna_name=antenna_name,
    )
    .squeeze(drop=True)
    .plot.line(x='baseband_frequency')
)
plt.subplots()
dataset.spectrogram_histogram.sel(
    spectrogram_power_bin=slice(-120, -60), center_frequency=fc
).plot.line(x='spectrogram_power_bin')
plt.subplots()
dataset.spectrogram.sel(center_frequency=fc, antenna_name=antenna_name).plot();